In [3]:
import re
import urllib
import numpy as np
import pandas as pd
from langdetect import detect

In [3]:
data_raw = pd.read_csv("data/allTweets.csv", index_col=0)
data_raw.head(5)
data_raw.drop('X3', axis=1, inplace=True)

In [62]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107056 entries, 1 to 107056
Data columns (total 4 columns):
X1    107055 non-null object
X2    107055 non-null object
X3    107055 non-null object
X4    107055 non-null object
dtypes: object(4)
memory usage: 4.1+ MB


In [4]:
data_raw.columns = ['User', 'Tweet', 'Datetime']
data_raw.head(5)

,User,Tweet,Datetime
1,@KueddeR,RT @Nebelspalter: Das gab es noch nie: Bundesk...,"May 23, 2015 at 06:05PM"
2,@silver_train,@diuuk Sicher nicht. Das #RTVG ist die falsche...,"May 23, 2015 at 06:12PM"
3,@peter_schibli,RT @rtvgja: Keine Billag-Schnüffler mehr? Ja b...,"May 23, 2015 at 06:25PM"
4,@peter_schibli,RT @rtvgja: Herr @F_Leutenegger : Distanzieren...,"May 23, 2015 at 06:25PM"
5,@bkuonen,"Jetzt wo ich weiss, dass CL, #superleague und ...","May 23, 2015 at 06:32PM"


In [5]:
data_raw.iloc[500:510]

,User,Tweet,URL,Datetime
501,@valabg,„Warum sollte ich 1 Bahnfahrkarte kaufen? Der ...,http://twitter.com/valabg/status/9253661018396...,"October 31, 2017 at 03:17PM"
502,@MikeBould3r,@NatalieRickli Arbeitet für eine TV-Werbeagent...,http://twitter.com/MikeBould3r/status/92536611...,"October 31, 2017 at 03:17PM"
503,@GyzTuufel,RT @peterhettich: Kollege Prof. Mark Schelker ...,http://twitter.com/GyzTuufel/status/9253661845...,"October 31, 2017 at 03:17PM"
504,@PeerStonebridge,RT @NZZ: #SRG-Mitarbeiter werfen ihren Chefs V...,http://twitter.com/PeerStonebridge/status/9253...,"October 31, 2017 at 03:21PM"
505,@karstenpater,RT @srgwatch: Die (Un-)Logik der Fernseh-Steue...,http://twitter.com/karstenpater/status/9253685...,"October 31, 2017 at 03:27PM"
506,@LahorJakrlin,@ZeitRauber @bglaettli @gebizzle ... oder sie ...,http://twitter.com/LahorJakrlin/status/9253686...,"October 31, 2017 at 03:27PM"
507,@ThomasLaeubli,@ZeitRauber @VinzenzWyss @FelixSchneuwly @thia...,http://twitter.com/ThomasLaeubli/status/925368...,"October 31, 2017 at 03:27PM"
508,@bdrtschr,RT @nachdenkend: Ein nicht unbekannter Werber ...,http://twitter.com/bdrtschr/status/92537060882...,"October 31, 2017 at 03:35PM"
509,@attila_gaspar,"RT @MarcBuergi: Liebe Freunde, Kolleginnen und...",http://twitter.com/attila_gaspar/status/925371...,"October 31, 2017 at 03:38PM"
510,@Kiamara91,RT @srgwatch: Die (Un-)Logik der Fernseh-Steue...,http://twitter.com/Kiamara91/status/9253715382...,"October 31, 2017 at 03:39PM"


In [7]:
data_raw.User.value_counts(dropna=False).head(20)

@sms2sms            1541
@MiniSchwiz         1487
@KarlMller13        1258
@RolfMll35367465    1185
@_macmike           1093
@HeinzLindenmann     894
@morvjn              833
@DailyTalk           805
@ProBillag           747
@tevau42             733
@person_tw           730
@LahorJakrlin        692
@scentedrebel        680
@SwissLibArmy        617
@bikejourno          615
@byebyebillag        610
@Voegizug            558
@wahlch15            550
@kindlimann          550
@NoSendeschluss      545
Name: User, dtype: int64

In [5]:
data_raw.Datetime.describe()
data_raw.Datetime = pd.to_datetime(data_raw.Datetime)
data_raw = data_raw.set_index('Datetime')
data_raw = data_raw['2017':'2018'] 

In [67]:
data_raw.head(5)

,User,Tweet
Datetime,,
2015-05-23 18:05:00,@KueddeR,RT @Nebelspalter: Das gab es noch nie: Bundesk...
2015-05-23 18:12:00,@silver_train,@diuuk Sicher nicht. Das #RTVG ist die falsche...
2015-05-23 18:25:00,@peter_schibli,RT @rtvgja: Keine Billag-Schnüffler mehr? Ja b...
2015-05-23 18:25:00,@peter_schibli,RT @rtvgja: Herr @F_Leutenegger : Distanzieren...
2015-05-23 18:32:00,@bkuonen,"Jetzt wo ich weiss, dass CL, #superleague und ..."


In [8]:
# Retweets markieren
check_for_retweets = lambda x: True if str(x)[:2] == 'RT' else False
data_raw['Retweet'] = data_raw['Tweet'].apply(check_for_retweets)

# Urheber von Original-Tweet ermitteln
find_tweet_orig = lambda x: str(x).split()[1][:-1] if str(x)[:2] == 'RT' else np.nan
data_raw['RT_from'] = data_raw['Tweet'].apply(find_tweet_orig)

# Tweets die keine Antworten oder Retweets sind markieren
check_orig_tweet = lambda x: True if str(x)[:2] != 'RT' and str(x)[:1] != '@' else False
data_raw['Orig_Tweet'] = data_raw['Tweet'].apply(check_orig_tweet)

# Alle Diskussionsteilnehmer (ausser Urheber von RTs) ermitteln
def find_tweet_participants(tweet):
    handle_list = re.findall(r'\@\w+', str(tweet))
    if str(tweet)[:2] == 'RT' and handle_list:
        del handle_list[0]
    return handle_list
data_raw['Tweet_Participants'] = data_raw['Tweet'].apply(find_tweet_participants)

# Vermutliche Replies markieren
check_reply = lambda x: True if str(x)[:1] == '@' else False
data_raw['Reply'] = data_raw['Tweet'].apply(check_reply)

# Reinen Text aus Tweet etrahieren
clean_tweet = lambda x: ' '.join([word for word in str(x).split() if word != 'RT' and bool(re.match('\@', word)) != True])
data_raw['Text'] = data_raw['Tweet'].apply(clean_tweet)

# Tags aus Tweet etrahieren
find_tweet_tags = lambda x: [tweet[1:] for tweet in re.findall(r'\#\w+', str(x))]
data_raw['Tags'] = data_raw['Tweet'].apply(find_tweet_tags)

# Links aus Tweets extrahieren
find_links = lambda x: re.findall(r"http\S+", x)
data_raw['Link'] = data_raw['Tweet'].apply(find_links)

In [8]:
# Sprache der Tweets ermitteln
def detect_language(text, tags):
        try:
            return detect(text + ' '.join(tags))
        except:
            print("Language cannot be recognised.")
            return False
data_raw['Lang'] = data_raw.apply(lambda row: detect_language(row['Text'], row['Tags']), axis=1)

Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language cannot be recognised.
Language

In [10]:
# Daten zwischenspeichern
data_raw.to_pickle('data/tweets.pkl')

In [28]:
# Daten einlesen
data_raw = pd.read_pickle('data/tweets.pkl')

In [11]:
data_raw.RT_from.value_counts(dropna=False).head(10)

NaN                 54799
@NoSendeschluss      1939
@operationlibero     1090
@NoBillag            1000
@person_tw            964
@ProBillag            866
@LahorJakrlin         801
@NoBillagNein         524
@EnneUlla             492
@redder66             481
Name: RT_from, dtype: int64

In [12]:
data_raw.Lang.value_counts(dropna=False)

de       87803
fr        7746
en        2121
tl        1932
it        1378
sv        1088
no         341
False      174
da         155
af         148
nl         135
so          59
ru          46
ca          42
hr          33
pt          33
es          31
tr          28
et          28
id          24
ro          18
fi          12
cy          10
sl           8
hu           8
bg           4
vi           4
pl           4
lt           2
el           2
sq           1
ur           1
lv           1
uk           1
ja           1
sw           1
sk           1
Name: Lang, dtype: int64

In [54]:
link_counter = 0
link_series = pd.Series()
def count_empty(links):
    global link_counter
    global link_series
    for link in links:
        link_counter += 1
        link_series.append(pd.Series([link]))
data_raw['Link'].apply(count_empty)
print(link_counter, "\n")
print(data_raw['Link'].count())

80928 

103424


In [55]:
link_series.head()

Series([], dtype: float64)

### Text hinter URLs scrapen

In [22]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# specify the url
quote_page = 'http://www.bloomberg.com/quote/SPX:IND'
with urllib.request.urlopen(quote_page) as response:
   html = response.read().decode('utf-8')

In [26]:
def remove_html_tags(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)
print(remove_html_tags(html))



  SPX Quote - S&amp;P 500 Index - Bloomberg Markets!function(e,t){function u(){return t.getElementById("bb-nav")}function M(){return document.querySelector(".bb-unsupported-message__text")}function L(){var e=document.getElementById("bb_unsupported_custom_message");if(e)return JSON.parse(e.innerHTML).message}function o(){var o='body.bb-unsupported-browser .bb-nav-placeholder{height:auto}body.bb-unsupported-browser #bb-that,body.bb-unsupported-browser .bb-nav-root{display:none;height:auto}body.bb-unsupported-browser .bb-nav-root{display:block}.bb-unsupported-message{background-color:#000;padding:20px 0}@media screen and (max-width: 63.6875rem){.bb-unsupported-message__logo-box{padding-bottom:10px}.bb-unsupported-message__copy{margin:0}}@media screen and (min-width: 63.75rem){.bb-unsupported-message__content{font-size:12px;display:table-cell;padding-left:50px}.bb-unsupported-message__logo-box{display:table-cell;vertical-align:middle}.bb-unsupported-message__copy{margin:1em 0}}.bb-unsupp

### Teil der Daten kodieren

In [9]:
from sklearn.model_selection import train_test_split
data_labeled, data_unlabeled  = train_test_split(data_raw, test_size=0.99)
data_labeled.shape

(1034, 10)

In [ ]:
def label_tweets(text):
    label_desc = "Codes: (1) Argument1 / (2) Argument2 / (3) Argument2/ (4) Argument2/ (5) Argument2\nFolgenden Tweet zu ordnen:\n'"
    code = input(label_desc + text + "':\n")
    return code
data_labeled['Label'] = data_labeled['Text'].apply(label_tweets)

Codes: (1) Argument1 / (2) Argument2 / (3) Argument2/ (4) Argument2/ (5) Argument2
Folgenden Tweet zu ordnen:
'Gut zu wissen: 13 private TV- und 22 Radio-Sender erhalten aktuell jährlich 67,5 Mio. Franken der #Billag-Gebühren - künftig werden es 81 Mio. Franken sein. Ein Wegfall dieser Gelder wäre für die meisten dieser Sender das Ende. #NeinzuNoBillag #Abst18':



test_tweet = "RT @LahorJakrlin: @SilFoiera @tagesanzeiger este kljsd flkajs dflaksdf @kdf alsdkjf alksdf"

In [6]:
data_raw['URL'].head()

KeyError: 'URL'

In [60]:
[word for word in str(test_tweet).split() if word != 'RT' and bool(re.match('\@', word)) != True]

['este', 'kljsd', 'flkajs', 'dflaksdf', 'alsdkjf', 'alksdf']

In [58]:
test_word = 'LahorJakrlin:'

In [59]:
bool(re.match('\@', test_word))

False

In [93]:
data_raw.head()

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang
Datetime,,,,,,,,,,
2015-05-23 18:05:00,@KueddeR,RT @Nebelspalter: Das gab es noch nie: Bundesk...,True,@Nebelspalter,False,[],False,"Das gab es noch nie: Bundeskanzlei druckt 5,35...","[RTVG, Billag]",de
2015-05-23 18:12:00,@silver_train,@diuuk Sicher nicht. Das #RTVG ist die falsche...,False,NaN,False,"[@diuuk, @Nullmedien]",True,Sicher nicht. Das #RTVG ist die falsche Vorlag...,[RTVG],de
2015-05-23 18:25:00,@peter_schibli,RT @rtvgja: Keine Billag-Schnüffler mehr? Ja b...,True,@rtvgja,False,[],False,Keine Billag-Schnüffler mehr? Ja bitte! #RTVGj...,"[RTVGja, RTVG, Abst15]",de
2015-05-23 18:25:00,@peter_schibli,RT @rtvgja: Herr @F_Leutenegger : Distanzieren...,True,@rtvgja,False,"[@F_Leutenegger, @derbund]",False,Herr : Distanzieren Sie sich von der niveaulos...,"[abst15, RTVG]",de
2015-05-23 18:32:00,@bkuonen,"Jetzt wo ich weiss, dass CL, #superleague und ...",False,NaN,True,[@srfsport],False,"Jetzt wo ich weiss, dass CL, #superleague und ...","[superleague, RTVG]",de


In [6]:
teststring = "kjs dfkjsdf jkahlsd ds"
testlist = ['test', 'test2', 'test3']
print(teststring + ' '.join(testlist))

kjs dfkjsdf jkahlsd dstest test2 test3


In [18]:
testinput = input("Hier input eingeben:\n")

Hier input eingeben:
Blabla


In [15]:
testinput

'blabla'

In [29]:
pd.DataFrame.drop?

In [31]:
data_raw['2015']

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang
Datetime,,,,,,,,,,
2015-05-23 18:05:00,@KueddeR,RT @Nebelspalter: Das gab es noch nie: Bundesk...,True,@Nebelspalter,False,[],False,"Das gab es noch nie: Bundeskanzlei druckt 5,35...","[RTVG, Billag]",de
2015-05-23 18:12:00,@silver_train,@diuuk Sicher nicht. Das #RTVG ist die falsche...,False,NaN,False,"[@diuuk, @Nullmedien]",True,Sicher nicht. Das #RTVG ist die falsche Vorlag...,[RTVG],de
2015-05-23 18:25:00,@peter_schibli,RT @rtvgja: Keine Billag-Schnüffler mehr? Ja b...,True,@rtvgja,False,[],False,Keine Billag-Schnüffler mehr? Ja bitte! #RTVGj...,"[RTVGja, RTVG, Abst15]",de
2015-05-23 18:25:00,@peter_schibli,RT @rtvgja: Herr @F_Leutenegger : Distanzieren...,True,@rtvgja,False,"[@F_Leutenegger, @derbund]",False,Herr : Distanzieren Sie sich von der niveaulos...,"[abst15, RTVG]",de
2015-05-23 18:32:00,@bkuonen,"Jetzt wo ich weiss, dass CL, #superleague und ...",False,NaN,True,[@srfsport],False,"Jetzt wo ich weiss, dass CL, #superleague und ...","[superleague, RTVG]",de
2015-05-23 19:25:00,@compusoft_,RT @hedilamnna: @JoeEsceha Die Weichen sind g...,True,@hedilamnna,False,"[@JoeEsceha, @NoBillag]",False,Die Weichen sind gestellt! Erst mal am 14.6. n...,[RTVG],de
2015-05-23 19:35:00,@Nullmedien,"Die #RTVG-Gegner weisen in die unheimliche, me...",False,NaN,True,[],False,"Die #RTVG-Gegner weisen in die unheimliche, me...",[RTVG],de
2015-05-23 19:55:00,@silver_train,RT @Nullmedien: Die #RTVG-Gegner weisen in die...,True,@Nullmedien,False,[],False,"Die #RTVG-Gegner weisen in die unheimliche, me...",[RTVG],de
2015-05-23 20:48:00,@iHaeck,RT @Nebelspalter: Das gab es noch nie: Bundesk...,True,@Nebelspalter,False,[],False,"Das gab es noch nie: Bundeskanzlei druckt 5,35...","[RTVG, Billag]",de
